In [9]:
"Columns"
ecs_a = ['o','p','q']
#sq_a = ['i']
hw_a = ['f','g','h','i','j','k','l']
hw_names = [1, 2 ,5 ,4 ,6 ,7 ,3 ]
hw_vals =  [20,22,15,15,15,15,25]
labs_a = ['m','n']

#convert column letters into values
alphabet = 'a.b.c.d.e.f.g.h.i.j.k.l.m.n.o.p.q.r.s.t.u.v.w.x.y.z'.split('.')
alphamap = {alphabet[i]:i for i in range(0,len(alphabet))}
tonum = lambda x: [alphamap[a] for a in x]


ecs_ids = tonum(ecs_a)#[alphamap[ec] for ec in ecs_a]
#sq_ids =  tonum(sq_a)
hw_ids = tonum(hw_a)

lab_ids = tonum(labs_a)
lab_vals = [15]*len(lab_ids)

In [17]:
import csv


def pingline(segment,idv):
    try: 
        x = float(segment[idv])
        return x
    except: return 0


def processLinetoStudent(line):
    segment = line
    student = {}
    student['name']=segment[0]
    student['ID']=segment[1]
    student['uid']=segment[2]
    student['login']=segment[3]
    student['section'] = segment[4]
    hws = []
    labs = []
    
    hws = [pingline(segment,idv) for idv in hw_ids] 
    labs = [pingline(segment,idv) for idv in lab_ids]
    ecs = [pingline(segment,idv) for idv in ecs_ids]
    
    #adding credit for HW6
    #hws.append(15)
    #hw_base = hw_vals+[15]
    
#     for i in range(5,16):
#         if i in [5,8,10,13,14]:
#             try:
#                 hws.append()
#             except:
#                 hws.append(0)
#         elif i in [6,7,11,12,15]:
#             try:
#                 labs.append(float(segment[i]))
#             except:
#                 labs.append(0)
    student['hw']=hws
    student['hw_base']=hw_vals
    student['labs']=labs
    student['lab_base']=lab_vals
    student['ecs'] = ecs
    return student




def grade(student):
    hws = student['hw'][:]
    hw_base = student['hw_base'][:]
    lowest_hw = min(hws)
    hw_i = hws.index(lowest_hw)
    print("Drop HW: ",hw_names[hw_i])
    hws.pop(hw_i)
    hw_base.pop(hw_i)
    
    #process EC
    ecs = student['ecs'][:]
    eca = [a for a in ecs if a>0]
    ecv = [15 for a in eca]
    hws = hws+ecs
    hw_base = hw_base + ecv
        
    
    num = sum(hws)
    denom = sum(hw_base)
    hw_percent = num/denom
    lab_percent = sum(student['labs'])/sum(student['lab_base'])
    
    
    
    grade = 0.6*hw_percent+0.4*lab_percent#0.025
    
    student['lab_percent']=lab_percent
    student['hw_percent']=hw_percent
    student['grade']=grade
    student['dropped_hw']=hw_names[hw_i]
    #student['gpa'] = scoretogpa_linear1(grade)
    p = score(grade)
    student['gpa'] = p
    return student

def gradedStudentToList(student):
    out=[student['name'],student['ID'],student['uid'],student['login'],student['section']]
    out = out + student['hw']
    out = out + student['labs']
    out = out + student['ecs']
    out = out + [student['dropped_hw']]
    out = out + [student['hw_percent'],student['lab_percent'],student['grade']]
    out = out + [student['gpa']]
    return out

def gradedStudentTitleList(hws,labs,ec):
    out =['name','ID','uid','login','section']
    out = out + [f"hw{x}" for x in range(1,len(hws)+1) ]
    out = out + [f"lab{x}" for x in range(1,len(labs)+1) ]
    out = out + [f"EC{x}" for x in range(1,len(ec)+1) ]
    out = out + ['dropped_hw']
    out = out + ['hw_percent','lab_percent','grade','gpa']
    print('///////////////////')
    print(out)
    print('///////////////////')
    return out



def score(p):
    percent=p+0.025
    maxpercent = 0.97
    minpercent = 0.60
    if percent<minpercent: return "0.0"
    if percent>=maxpercent: return "4.0"
    val = 4.0*(percent-minpercent)/(maxpercent-minpercent)
    return "{0:0.1f}".format(val)


GPA_MAP = [ (0.60,0,'X'),
        (0.60, 1.0,'D-'),(0.66, 1.0,'D'), (0.69, 1.3,"D+"),
        (0.73, 2.0,"C-"),(0.76, 2.0,'C'),(0.79, 2.3,'C+'),
        (0.83, 2.7,"B-"),(0.86, 3.0,'B'),(0.89, 3.3,'B+'),
        (0.93, 3.7,"A-"),(0.96, 3.9,'A'),(1.0, 4,'A+') ]
BREAKPOINTS, GPAS, LETTERS = zip(*GPA_MAP)

def scoretogpa_linear1(score):
    for index, breakpoint in enumerate(BREAKPOINTS):
        if score < breakpoint:
            return GPA_MAP[index]
    return GPA_MAP[index] 


def gradeClass(file):
    with open(file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        students = []
        for row in csv_reader:
            if line_count <2:
                print(row)
                line_count+=1
            else:
                student = processLinetoStudent(row)
                #print(student)
                student = grade(student)
                students.append(student)
                #print(student)
    print('Done grading')

    with open("graded.csv",'w',newline='\n', encoding='utf-8') as write_file:
        writer = csv.writer(write_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        
        writer.writerow(gradedStudentTitleList(hw_ids,lab_ids,ecs_ids))
        
        for student in students:
            writer.writerow(gradedStudentToList(student))
    print('Done writing')

In [18]:
gradeClass("grades-base.csv")

['Student', 'ID', 'SIS User ID', 'SIS Login ID', 'Section', 'Numbers (1975224)', 'HW2: Ready Set Iterate (1975225)', 'HW5:  Classy Classes (1975227)', 'HW4: Data Must Flow (1975226)', 'HW6 : Arrays and CSG (1975229)', 'HW 7: Beyond Jupyter (1975230)', 'HW3: Simulate and Analyze (2027394)', 'Lab: Spirograph G-code generator (1975233)', 'Lab 2: Slicers (1975234)', 'Extra Credit 1 - Longitude (1975220)', 'Extra Credit 2 - Player Piano (1975221)', 'Extra Credit 3 - How Innovation Works (1975222)']
['    Points Possible', '', '', '', '', '20', '22', '15', '15', '15', '15', '25', '15', '15', '15', '15', '15']
Drop HW:  5
Drop HW:  5
Drop HW:  4
Drop HW:  5
Drop HW:  5
Drop HW:  4
Drop HW:  4
Drop HW:  5
Drop HW:  6
Drop HW:  5
Drop HW:  5
Drop HW:  6
Drop HW:  6
Drop HW:  5
Drop HW:  6
Drop HW:  3
Drop HW:  5
Drop HW:  5
Drop HW:  2
Done grading
///////////////////
['name', 'ID', 'uid', 'login', 'section', 'hw1', 'hw2', 'hw3', 'hw4', 'hw5', 'hw6', 'hw7', 'lab1', 'lab2', 'EC1', 'EC2', 'EC3', 